In [1]:
#APP NAME FlixTube

In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [2]:
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "/home/lyle/Downloads/credentials.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("/home/lyle/token.pickle"):
        with open("/home/lyle/token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("/home/lyle/token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

In [3]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [4]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [5]:
def print_video_infos(video_response):
    items = video_response.get("items")[0]
    # get the snippet, statistics & content details from the video response
    snippet         = items["snippet"]
    statistics      = items["statistics"]
    content_details = items["contentDetails"]
    # get infos from the snippet
    channel_title = snippet["channelTitle"]
    title         = snippet["title"]
    description   = snippet["description"]
    publish_time  = snippet["publishedAt"]
    # get stats infos
    comment_count = statistics["commentCount"]
    like_count    = statistics["likeCount"]
    view_count    = statistics["viewCount"]
    # get duration from content details
    duration = content_details["duration"]
    # duration in the form of something like 'PT5H50M15S'
    # parsing it to be something like '5:50:15'
    parsed_duration = re.search(f"PT(\d+H)?(\d+M)?(\d+S)", duration).groups()
    duration_str = ""
    for d in parsed_duration:
        if d:
            duration_str += f"{d[:-1]}:"
    duration_str = duration_str.strip(":")
    print(f"""\
    Title: {title}
    Description: {description}
    Channel Title: {channel_title}
    Publish time: {publish_time}
    Duration: {duration_str}
    Number of comments: {comment_count}
    Number of likes: {like_count}
    Number of views: {view_count}
    """)

In [6]:
video_url = "https://www.youtube.com/watch?v=jNQXAC9IVRw&ab_channel=jawed"
# parse video ID from URL
video_id = get_video_id_by_url(video_url)
# make API call to get video info
response = get_video_details(youtube, id=video_id)
# print extracted video infos
print_video_infos(response)

    Title: Me at the zoo
    Description: 
    Channel Title: jawed
    Publish time: 2005-04-24T03:31:52Z
    Duration: 19
    Number of comments: 11269539
    Number of likes: 13240721
    Number of views: 257550989
    


In [7]:
video_url = "https://www.youtube.com/watch?v=hG8KEfG5lwg"
# parse video ID from URL
video_id = get_video_id_by_url(video_url)
# make API call to get video info
response = get_video_details(youtube, id=video_id)
# print extracted video infos
print_video_infos(response)

    Title: Gulag - The Story | Part 2: Propagation - 1934 - 1945 | Free Documentary History
    Description: Gulag - The Story - Part 2: Propagation - 1934 - 1945 | History Documentary

Watch 'Gulag  - The Story: Part 3' here: https://youtu.be/dBjcT0QxSS0

A major political, historical, human and economic fact of the 20th century, the Gulag, the extremely punitive Soviet concentration camp system, remains largely unknown.
The history of the Gulag is long, complex and in many ways out of the ordinary. From the Revolution of 1917 to Gorbachev, touching on the civil war, the Great Terror, World War II, the Cold War and the death of Stalin, this series describes the workings of the Gulag.
How and why did the USSR create this system of forced- labour camps in which 20 million prisoners were exploited and worked to the bone? 
Through the exceptional fates of numerous protagonists, both executioners and victims, the history of the Gulag is deciphered with previously-unreleased documentary sou

In [8]:
response.keys()

dict_keys(['kind', 'etag', 'items', 'pageInfo'])

In [9]:
response['kind']

'youtube#videoListResponse'

In [10]:
response['etag']

'M2uNW3nunQf0rcAEPjUao1ycwQI'

In [11]:
type(response['items'])

list

In [12]:
response['items']

[{'kind': 'youtube#video',
  'etag': 'x58yQDpFQiymHTQgbHv5oyi3O-0',
  'id': 'hG8KEfG5lwg',
  'snippet': {'publishedAt': '2023-02-16T23:00:52Z',
   'channelId': 'UCsgPO6cNV0wBG-Og3bUZoFA',
   'title': 'Gulag - The Story | Part 2: Propagation - 1934 - 1945 | Free Documentary History',
   'description': "Gulag - The Story - Part 2: Propagation - 1934 - 1945 | History Documentary\n\nWatch 'Gulag  - The Story: Part 3' here: https://youtu.be/dBjcT0QxSS0\n\nA major political, historical, human and economic fact of the 20th century, the Gulag, the extremely punitive Soviet concentration camp system, remains largely unknown.\nThe history of the Gulag is long, complex and in many ways out of the ordinary. From the Revolution of 1917 to Gorbachev, touching on the civil war, the Great Terror, World War II, the Cold War and the death of Stalin, this series describes the workings of the Gulag.\nHow and why did the USSR create this system of forced- labour camps in which 20 million prisoners were exp

In [14]:
response['pageInfo']

{'totalResults': 1, 'resultsPerPage': 1}

In [15]:
type(response['items'][0])

dict

In [16]:
response['items'][0]

{'kind': 'youtube#video',
 'etag': 'x58yQDpFQiymHTQgbHv5oyi3O-0',
 'id': 'hG8KEfG5lwg',
 'snippet': {'publishedAt': '2023-02-16T23:00:52Z',
  'channelId': 'UCsgPO6cNV0wBG-Og3bUZoFA',
  'title': 'Gulag - The Story | Part 2: Propagation - 1934 - 1945 | Free Documentary History',
  'description': "Gulag - The Story - Part 2: Propagation - 1934 - 1945 | History Documentary\n\nWatch 'Gulag  - The Story: Part 3' here: https://youtu.be/dBjcT0QxSS0\n\nA major political, historical, human and economic fact of the 20th century, the Gulag, the extremely punitive Soviet concentration camp system, remains largely unknown.\nThe history of the Gulag is long, complex and in many ways out of the ordinary. From the Revolution of 1917 to Gorbachev, touching on the civil war, the Great Terror, World War II, the Cold War and the death of Stalin, this series describes the workings of the Gulag.\nHow and why did the USSR create this system of forced- labour camps in which 20 million prisoners were exploited 

In [11]:
def search(youtube, **kwargs):
    return youtube.search().list(
        part="snippet",
        **kwargs
    ).execute()

In [12]:
# search for the query 'python' and retrieve 2 items only
response = search(youtube, q="python", maxResults=2)
items = response.get("items")
for item in items:
    # get the video ID
    video_id = item["id"]["videoId"]
    # get the video details
    video_response = get_video_details(youtube, id=video_id)
    # print the video details
    print_video_infos(video_response)
    print("="*50)

    Title: Python Tutorial - Python Full Course for Beginners
    Description: Python tutorial - Python full course for beginners - Go from Zero to Hero with Python (includes machine learning & web development projects).

🔥 Want to master Python? Get my Python mastery course: http://bit.ly/35BLHHP
👍 Subscribe for more Python tutorials like this: https://goo.gl/6PYaGF

👉 Watch the new edition: https://youtu.be/kqtD5dpn9C8

📕 Get my FREE Python cheat sheet: http://bit.ly/2Gp80s6

Want to learn more from me? 

Courses: https://codewithmosh.com
Twitter: https://twitter.com/moshhamedani
Facebook: https://www.facebook.com/programmingwithmosh/
Blog: http://programmingwithmosh.com

#Python, #MachineLearning, #WebDevelopment

🔗 Supplementary Materials (Spreadsheet): https://bit.ly/3cb2YNo

📔 Python Exercises for Beginners: https://goo.gl/1XnQB1

⭐ My Favorite Python Books
- Python Crash Course: https://amzn.to/2GqMdjG
- Automate the Boring Stuff with Python: https://amzn.to/2N71d6S
- A Smarter 

In [13]:
def parse_channel_url(url):
    """
    This function takes channel `url` to check whether it includes a
    channel ID, user ID or channel name
    """
    path = p.urlparse(url).path
    id = path.split("/")[-1]
    if "/c/" in path:
        return "c", id
    elif "/channel/" in path:
        return "channel", id
    elif "/user/" in path:
        return "user", id

def get_channel_id_by_url(youtube, url):
    """
    Returns channel ID of a given `id` and `method`
    - `method` (str): can be 'c', 'channel', 'user'
    - `id` (str): if method is 'c', then `id` is display name
        if method is 'channel', then it's channel id
        if method is 'user', then it's username
    """
    # parse the channel URL
    method, id = parse_channel_url(url)
    if method == "channel":
        # if it's a channel ID, then just return it
        return id
    elif method == "user":
        # if it's a user ID, make a request to get the channel ID
        response = get_channel_details(youtube, forUsername=id)
        items = response.get("items")
        if items:
            channel_id = items[0].get("id")
            return channel_id
    elif method == "c":
        # if it's a channel name, search for the channel using the name
        # may be inaccurate
        response = search(youtube, q=id, maxResults=1)
        items = response.get("items")
        if items:
            channel_id = items[0]["snippet"]["channelId"]
            return channel_id
    raise Exception(f"Cannot find ID:{id} with {method} method")

In [14]:
def get_channel_videos(youtube, **kwargs):
    return youtube.search().list(
        **kwargs
    ).execute()


def get_channel_details(youtube, **kwargs):
    return youtube.channels().list(
        part="statistics,snippet,contentDetails",
        **kwargs
    ).execute()

In [15]:
channel_url = "https://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ"
# get the channel ID from the URL
channel_id = get_channel_id_by_url(youtube, channel_url)
# get the channel details
response = get_channel_details(youtube, id=channel_id)
# extract channel infos
snippet = response["items"][0]["snippet"]
statistics = response["items"][0]["statistics"]
channel_country = snippet["country"]
channel_description = snippet["description"]
channel_creation_date = snippet["publishedAt"]
channel_title = snippet["title"]
channel_subscriber_count = statistics["subscriberCount"]
channel_video_count = statistics["videoCount"]
channel_view_count  = statistics["viewCount"]
print(f"""
Title: {channel_title}
Published At: {channel_creation_date}
Description: {channel_description}
Country: {channel_country}
Number of videos: {channel_video_count}
Number of subscribers: {channel_subscriber_count}
Total views: {channel_view_count}
""")
# the following is grabbing channel videos
# number of pages you want to get
n_pages = 2
# counting number of videos grabbed
n_videos = 0
next_page_token = None
for i in range(n_pages):
    params = {
        'part': 'snippet',
        'q': '',
        'channelId': channel_id,
        'type': 'video',
    }
    if next_page_token:
        params['pageToken'] = next_page_token
    res = get_channel_videos(youtube, **params)
    channel_videos = res.get("items")
    for video in channel_videos:
        n_videos += 1
        video_id = video["id"]["videoId"]
        # easily construct video URL by its ID
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_response = get_video_details(youtube, id=video_id)
        print(f"================Video #{n_videos}================")
        # print the video details
        print_video_infos(video_response)
        print(f"Video URL: {video_url}")
        print("="*40)
    print("*"*100)
    # if there is a next page, then add it to our parameters
    # to proceed to the next page
    if "nextPageToken" in res:
        next_page_token = res["nextPageToken"]


Title: freeCodeCamp.org
Published At: 2014-12-16T21:18:48Z
Description: Learn to code for free.
Country: US
Number of videos: 1408
Number of subscribers: 7230000
Total views: 496756910

================Video #1================
    Title: Passwords in HTML #shorts
    Description: Learn more about HTML with this crash course: https://www.youtube.com/watch?v=916GWv2Qs08
    Channel Title: freeCodeCamp.org
    Publish time: 2023-02-03T16:37:12Z
    Duration: 55
    Number of comments: 26
    Number of likes: 1991
    Number of views: 32656
    
Video URL: https://www.youtube.com/watch?v=1hG0i9dQXeg
================Video #2================
    Title: Webhooks Explained with a Roadtrip #shorts
    Description: Watch the full course: https://www.youtube.com/watch?v=41NOoEz3Tzc
    Channel Title: freeCodeCamp.org
    Publish time: 2023-01-24T17:11:22Z
    Duration: 59
    Number of comments: 8
    Number of likes: 892
    Number of views: 23201
    
Video URL: https://www.youtube.com/watch?v

================Video #8================
    Title: How do loops work in Python?
    Description: Watch the full Python course: https://www.youtube.com/watch?v=eWRfhZUzrAc
    Channel Title: freeCodeCamp.org
    Publish time: 2023-02-09T14:16:24Z
    Duration: 56
    Number of comments: 10
    Number of likes: 1095
    Number of views: 23128
    
Video URL: https://www.youtube.com/watch?v=WYLjcXpxFOQ
================Video #9================
    Title: Intro to Object Oriented Programming - Crash Course
    Description: Learn the basics of object-oriented programming all in one video.

✏️ Course created by Steven from NullPointer Exception. Check out their channel: https://www.youtube.com/channel/UCmWDlvMYYEbW42B8JyxFBcA

🎥 Introduction to Programming: https://www.youtube.com/watch?v=zOjov-2OZ0E

⭐️ Course Contents ⭐️
⌨️ (00:00) Introduction
⌨️ (07:37) Encapsulation
⌨️ (12:45) Abstraction
⌨️ (17:49) Inheritance
⌨️ (22:47) Polymorphism

⭐️ Sources ⭐️
🔗 https://docs.oracle.com/javase/tuto

AttributeError: 'NoneType' object has no attribute 'groups'

In [17]:
def get_comments(youtube, **kwargs):
    return youtube.commentThreads().list(
        part="snippet",
        **kwargs
    ).execute()

In [19]:
params = {
        'videoId': video_id, 
        'maxResults': 2,
        'order': 'relevance', # default is 'time' (newest)
    }
response = get_comments(youtube, **params)
items = response.get("items")

In [21]:
items

[{'kind': 'youtube#commentThread',
  'etag': '6fZWwTEJVQh_OWSOatLL9UkBAI8',
  'id': 'Ugw6vaP8mLYDFGbszOZ4AaABAg',
  'snippet': {'videoId': 'hG8KEfG5lwg',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': 'vLkH0krzFeSdPuXzYNLlUYqgLNk',
    'id': 'Ugw6vaP8mLYDFGbszOZ4AaABAg',
    'snippet': {'videoId': 'hG8KEfG5lwg',
     'textDisplay': 'Through the exceptional fates of numerous protagonists, both executioners, and victims, the history of the Gulag is deciphered with previously-unreleased documentary sources and the help of renowned historians and Gulag experts.<br>Episode 2:<br>Stalin, who was highly praised at the 17th congress of the Communist Party in 1934, launched the construction of the Moscow canal and a new Trans-Siberian route. The NKVD, which took over from the GPU, increased the number of camps and transformed the Gulag into a veritable prison industry. In 1935 the number of prisoners in the Gulag exceeded one million. The trials held by Moscow, which were the sho

In [22]:
for item in items:
    comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
    updated_at = item["snippet"]["topLevelComment"]["snippet"]["updatedAt"]
    like_count = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
    comment_id = item["snippet"]["topLevelComment"]["id"]
    print(f"""\
    Comment: {comment}
    Likes: {like_count}
    Updated At: {updated_at}
    ==================================\
    """)

    Comment: Through the exceptional fates of numerous protagonists, both executioners, and victims, the history of the Gulag is deciphered with previously-unreleased documentary sources and the help of renowned historians and Gulag experts.<br>Episode 2:<br>Stalin, who was highly praised at the 17th congress of the Communist Party in 1934, launched the construction of the Moscow canal and a new Trans-Siberian route. The NKVD, which took over from the GPU, increased the number of camps and transformed the Gulag into a veritable prison industry. In 1935 the number of prisoners in the Gulag exceeded one million. The trials held by Moscow, which were the showcase of the Great Purge, hid the repression that was hitting Soviet society and the anonymous mass executions and arbitrary arrests rapidly increased in number. In January 1939 2 million prisoners were working in the Gulag, but on 22 June 1941 Germany attacked the Soviet Union. As a consequence, in 1942, detention conditions in the Gu

In [17]:
# URL can be a channel or a video, to extract comments
url = "https://www.youtube.com/watch?v=jNQXAC9IVRw&ab_channel=jawed"
if "watch" in url:
    # that's a video
    video_id = get_video_id_by_url(url)
    params = {
        'videoId': video_id, 
        'maxResults': 2,
        'order': 'relevance', # default is 'time' (newest)
    }
else:
    # should be a channel
    channel_id = get_channel_id_by_url(url)
    params = {
        'allThreadsRelatedToChannelId': channel_id, 
        'maxResults': 2,
        'order': 'relevance', # default is 'time' (newest)
    }
# get the first 2 pages (2 API requests)
n_pages = 2
for i in range(n_pages):
    # make API call to get all comments from the channel (including posts & videos)
    response = get_comments(youtube, **params)
    items = response.get("items")
    # if items is empty, breakout of the loop
    if not items:
        break
    for item in items:
        comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        updated_at = item["snippet"]["topLevelComment"]["snippet"]["updatedAt"]
        like_count = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]
        comment_id = item["snippet"]["topLevelComment"]["id"]
        print(f"""\
        Comment: {comment}
        Likes: {like_count}
        Updated At: {updated_at}
        ==================================\
        """)
    if "nextPageToken" in response:
        # if there is a next page
        # add next page token to the params we pass to the function
        params["pageToken"] =  response["nextPageToken"]
    else:
        # must be end of comments!!!!
        break
    print("*"*70)

        Comment: We&#39;re so honored that the first ever YouTube video was filmed here!
        Likes: 2759398
        Updated At: 2020-02-17T18:58:15Z
        Comment: Hoy se cumplen 18 años de esta joya. Quien más esta aquí reviviendo el pasado?
        Likes: 429
        Updated At: 2023-02-14T06:15:32Z
**********************************************************************
        Comment: It&#39;s incredible to see how far youtube has advanced and how much the world has changed since this video was first uploaded. It serves as a reminder of just how far we have come and how much more is yet to come in the future. Thank you Jawed for capturing this moment in time and preserving it for future generations.🎉
        Likes: 1236
        Updated At: 2023-02-13T12:09:52Z
        Comment: It&#39;s insane that a 20 second video managed to change the world forever, and today we have this amazing website on the internet called YouTube.
        Likes: 151
        Updated At: 2023-01-22T22:38: